In [4]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 100)

In [5]:
# import discover data, and filter for list of emails according to their earliest use of discover
discover_data = pd.read_csv("discover_user_input_results.csv")
discover_first = discover_data.drop_duplicates(subset = ['email'], keep='first')
discover_first['timestamp2'] = pd.to_datetime(discover_first['timestamp'].map(lambda x: x.replace('th', "")))
discover_first.drop('timestamp', axis=1,inplace=True)
#discover_first.to_csv('discover_first.csv')


C:\Users\limzi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\limzi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [55]:
# Import shopify sales data
sales_data = pd.read_csv("shopify_orders_export_20180207.csv", 
                         low_memory=False, 
                         parse_dates=['Paid at', 'Fulfilled at', 'Created at'])

sales_data_clean = sales_data.drop(sales_data.columns.to_series()[-11:-1], axis=1)
# remove in-store purchases by dropping NA rows in Email column
sales_data_clean.dropna(subset=['Email'], axis=0, inplace=True)

# add columns referencing each customer's FIRST use of discover
sales_data_clean['discover_first_date'] = sales_data_clean['Email'].map(discover_first.set_index('email')['timestamp2'])
sales_data_clean['used_discover_already'] = (sales_data_clean['Created at']> sales_data_clean['discover_first_date']).map({True: "Used Discover", False: "Not yet"})
sales_data_clean['discover_sales_lead_time'] = sales_data_clean['Created at'] - sales_data_clean['discover_first_date']

# create sales dataset filtering customers who had used discover
# regardless of whether they are existing or completely new customers
post_discover_sales = sales_data_clean[sales_data_clean['used_discover_already'] == "Used Discover"]

# create sales dataset filtering customers who have never used discover and
# and also those customers' purchase histories right before they use discover
pre_discover_sales = sales_data_clean[~(sales_data_clean['used_discover_already'] == "Used Discover")]


In [65]:
post_discover_sales

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Phone,discover_first_date,used_discover_already,discover_sales_lead_time
0,191569914712,gilly.glanville@me.com,paid,2018-02-08 03:59:00,fulfilled,2018-02-08 03:59:01,yes,SGD,290.0,0.0,0.0,290.0,5OFFe53b8c2fb9df,5.0,NaN,2018-02-08 03:59:00,1,Resist Skin Restoring Moisturizer SPF 50 - 60ml,48.0,0.0,7970,True,False,fulfilled,Gilly Glanville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,External Credit,c563530104851.1,0.0,Paula's Choice,0.0,Jeremy Tan,Beauty Collective,9.0,1.811065e+11,NaN,Low,pos,0.0,NaN,NaT,Not yet,NaT
1,191569914712,gilly.glanville@me.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-08 03:59:00,1,Resist Optimal Results Hydrating Cleanser - 19...,36.0,0.0,7600,True,False,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT,Not yet,NaT
2,191569914712,gilly.glanville@me.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-08 03:59:00,1,Resist Advanced Smoothing Treatment 10% AHA (G...,55.0,0.0,7651,True,True,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT,Not yet,NaT
3,191569914712,gilly.glanville@me.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-08 03:59:00,1,Resist C15 Super Booster - 20 ml,68.0,0.0,7770,True,False,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT,Not yet,NaT
4,191569914712,gilly.glanville@me.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-08 03:59:00,1,Clinical Ceramide-Enriched Firming Moisturizer,88.0,0.0,2120,True,False,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaT,Not yet,NaT
5,191569914711,stephaniedata@yahoo.com,paid,2018-02-08 03:39:25,unfulfilled,NaT,no,SGD,38.0,5.5,0.0,43.5,5OFF5359ca0ded5b,5.0,2 Working-Day Delivery,2018-02-08 03:39:25,1,Clear Acne Extra Strength Exfoliating Treatmen...,43.0,NaN,6210,True,False,pending,Stephanie Ng,"6 Napier Road 07-05, Gleneagles Medical Centre...",6 Napier Road 07-05,Gleneagles Medical Centre 07-05,The Dermatology Practice at Gleneagles,Singapore,'258499,NaN,SG,9001 9072,Stephanie Ng,"6 Napier Road 07-05, Gleneagles Medical Centre...",6 Napier Road 07-05,Gleneagles Medical Centre 07-05,The Dermatology Practice at Gleneagles,Singapore,'258499,NaN,SG,9001 9072,NaN,NaN,NaN,Stripe,c531555713043.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,1.810939e+11,NaN,Low,web,0.0,6.590019e+09,NaT,Not yet,NaT
6,191569914710,hsmeaton@hotmail.com,paid,2018-02-08 00:42:57,unfulfilled,NaT,yes,SGD,48.0,9.0,0.0,57.0,NaN,0.0,Free Shipping above S$99 (ex.Taxes & Fees),2018-02-08 00:42:56,1,Calm Sensitive Daytime Moisturizer SPF 30 (Nor...,48.0,NaN,9130,True,False,pending,Heather Smeaton,"C3-4-1 Pantai Hillpark Phase 1, Jln Pantai Dalam",C3-4-1 Pantai Hillpark Phase 1,Jln Pantai Da

In [53]:
# create sales dataset filtering NEW customers who HAD NOT purchased prior to trying out discover
## obtain the transactions of new customers acquired after Discover launch who have used discover
post_discover_new_customers = post_discover_sales[~(post_discover_sales['Email'].isin(pre_discover_sales['Email'])) 
                                                  #&
                                                  #(pd.isnull(post_discover_sales['discover_first_date']) == False)
                                                 ]
post_discover_new_customers_unique = post_discover_new_customers[['Email', 'Created at']].drop_duplicates()
post_discover_new_customers_unique.sort_values('Created at', inplace=True)
post_discover_new_customers_unique['Previous Transaction'] = post_discover_new_customers_unique.groupby(['Email'])['Created at'].shift()
post_discover_new_customers_unique['Days Between'] = post_discover_new_customers_unique['Created at'] - post_discover_new_customers_unique['Previous Transaction']
post_discover_new_customers_unique['Days Between Int'] = (post_discover_new_customers_unique['Days Between'].dropna()/ np.timedelta64(1, "D")).astype(int)
post_discover_new_customers_unique['Buy Count'] = post_discover_new_customers_unique.groupby(['Email'])['Created at'].cumcount()+1
post_discover_new_customers_unique['Days Between'].mean()
# post_discover_new_customers_unique.to_excel("post_discover_new_customers_unique.xlsx")

In [86]:
# To validate charmander
post_discover_new_customers_validate = sales_data_clean[~(sales_data_clean['Email'].isin(post_discover_new_customers['Email']))]
#post_discover_sales[~(post_discover_sales['Email'].isin(pre_discover_sales['Email']))]
post_discover_new_customers_validate[post_discover_new_customers_validate['Created at'] >= "2017-09-09"]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Phone,discover_first_date,used_discover_already,discover_sales_lead_time
10,191569914707,jglyj82@gmail.com,paid,2018-02-07 13:02:33,unfulfilled,NaT,yes,SGD,82.0,0.0,0.0,82.0,NaN,0.0,Free Delivery (2 Working Days),2018-02-07 13:02:33,1,Skin Balancing Oil-Reducing Cleanser - 237 ml,34.0,0.0,1150,True,False,pending,Joanne Tan,13367 Holly Oak Cir,13367 Holly Oak Cir,NaN,NaN,Cerritos,'90703,CA,US,(562) 404-7973,Joanne Tan,"23 Stevens Drive #03-01, Parc Stevens",23 Stevens Drive #03-01,Parc Stevens,NaN,Singapore,'257914,NaN,SG,9109 2164,NaN,NaN,NaN,Stripe,c561204592659.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,1.801691e+11,NaN,Low,web,0.0,NaN,2018-02-07 12:54:54,Used Discover,0 days 00:07:39
11,191569914707,jglyj82@gmail.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-07 13:02:33,1,Resist Youth-Extending Daily Hydrating Fluid S...,48.0,0.0,7800,True,False,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2018-02-07 12:54:54,Used Discover,0 days 00:07:39
22,191569914-1758,zarr.gyii@gmail.com,paid,2018-02-07 11:16:46,fulfilled,2018-02-07 11:16:46,yes,SGD,29.0,0.0,0.0,29.0,NaN,0.0,NaN,2018-02-07 11:16:45,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,13.0,NaN,2017,True,False,fulfilled,Zarli Aung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stripe,c561033281555.1,0.0,Paula's Choice,0.0,Eli Goh,Front Porch,14.0,1.801029e+11,NaN,Low,pos,0.0,NaN,2017-09-26 03:45:14,Used Discover,134 days 07:31:31
23,191569914-1758,zarr.gyii@gmail.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-07 11:16:45,1,Clinical 1% Retinol Treatment - 5 ml,16.0,NaN,8017,True,False,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2017-09-26 03:45:14,Used Discover,134 days 07:31:31
29,191569914-1755,ycobonpue@gmail.com,paid,2018-02-07 08:34:54,fulfilled,2018-02-07 08:31:50,yes,SGD,59.0,0.0,0.0,59.0,NaN,0.0,NaN,2018-02-07 08:31:50,1,Resist Omega+ Complex - 30ml,59.0,NaN,2130,True,False,fulfilled,Yvonne Cobonpue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gift Card + Stripe,c560797745171.2,0.0,Paula's Choice,0.0,Eli Goh,Front Porch,14.0,1.800194e+11,NaN,Low,pos,0.0,NaN,2017-10-31 06:06:57,Used Discover,99 days 02:24:53
34,191569914693,karenkhor27@gmail.com,paid,2018-02-07 07:15:40,fulfilled,2018-02-07 12:28:57,yes,SGD,171.0,0.0,0.0,171.0,e1567fa9e6db,25.0,Free Delivery (2 Working Days),2018-02-07 07:15:40,1,Clear Acne Regular Strength Cream 2.5% BP - 67 ml,29.0,NaN,6100,True,False,fulfilled,Karen Khor,"08 Commonwealth Lane, #04-04",08 Commonwealth Lane,#04-04,G4S Secure Solutions Singapore Pte. Ltd.,Singapore,'149555,NaN,SG,+6596656577,Karen Khor,"08 Commonwealth Lane, #04-04",08 Commonwealth Lane,#04-04,G4S Secure Solutions Singapore Pte. Ltd.,Singapore,'149555,NaN,SG,+6596656577,NaN,NaN,NaN,Stripe,c560624992275.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,1.799875e+11,NaN,Low,web,0.0,NaN,2017-09-13 02:03:15,Used

In [96]:
pre_discover_new_customers = sales_data_clean[~(sales_data_clean['Email'].isin(post_discover_new_customers['Email']))]
pre_discover_new_customers_unique = pre_discover_new_customers[['Email', 'Created at']].drop_duplicates()
pre_discover_new_customers_unique.sort_values('Created at', inplace=True)
pre_discover_new_customers_unique['Previous Transaction'] = pre_discover_new_customers_unique.groupby(['Email'])['Created at'].shift()
pre_discover_new_customers_unique['Days Between'] = pre_discover_new_customers_unique['Created at'] - pre_discover_new_customers_unique['Previous Transaction']
pre_discover_new_customers_unique['Days Between Int'] = (pre_discover_new_customers_unique['Days Between'].dropna()/ np.timedelta64(1, "D")).astype(int)
pre_discover_new_customers_unique['Buy Count'] = pre_discover_new_customers_unique.groupby(['Email'])['Created at'].cumcount()+1
pre_discover_new_customers_unique['Days Between'].mean()
#pre_discover_new_customers_unique.to_excel("pre_discover_new_customers_unique.xlsx")

In [94]:
# just to validate bulbasaur calculation
pre_discover_new_customers_unique_validate = pre_discover_new_customers[['Email', 'Created at']].drop_duplicates().sort_values('Created at')
pre_discover_new_customers_unique_validate['Previous Transaction'] = pre_discover_new_customers_unique_validate.groupby(['Email'])['Created at'].shift()
pre_discover_new_customers_unique_validate['Days Between'] = pre_discover_new_customers_unique_validate['Created at'] - pre_discover_new_customers_unique_validate['Previous Transaction']
pre_discover_new_customers_unique_validate['Days Between Int'] = (pre_discover_new_customers_unique_validate['Days Between'].dropna()/ np.timedelta64(1, "D")).astype(int)
pre_discover_new_customers_unique_validate['Buy Count'] = pre_discover_new_customers_unique_validate.groupby(['Email'])['Created at'].cumcount()+1
pre_discover_new_customers_unique_validate['Days Between'].mean()
pre_discover_new_customers_unique_validate

,Email,Created at,Previous Transaction,Days Between,Days Between Int,Buy Count
29702,thanhvan84@gmail.com,2016-12-02 07:12:18,NaT,NaT,NaN,1
29699,joyesingapore@hotmail.com,2016-12-02 08:25:54,NaT,NaT,NaN,1
29696,trixtrixtrix@gmail.com,2016-12-02 08:40:12,NaT,NaT,NaN,1
29692,thetang34@hotmail.com,2016-12-02 09:03:53,NaT,NaT,NaN,1
29652,tottiroti@gmail.com,2016-12-02 15:23:05,NaT,NaT,NaN,1
29631,maitantitham@hotmail.com,2016-12-03 03:23:16,NaT,NaT,NaN,1
29562,ong_hui_yi@hotmail.com,2016-12-03 16:39:30,NaT,NaT,NaN,1
29555,hazelwongshihui@gmail.com,2016-12-04 03:01:21,NaT,NaT,NaN,1
29544,riaamoorthy@gmail.com,2016-12-04 05:01:41,NaT,NaT,NaN,1
29537,ana_adl27@hotmail.com,2016-12-04 07:01:10,NaT,NaT,NaN,1


In [99]:

# obtain the transactions of new customers acquired after Discover launch who have NOT used discover
'''
post_non_discover_new_customers = post_discover_sales[(~post_discover_sales['Email'].isin(pre_discover_sales['Email'])) 
                                                  &
                                                  (pd.isnull(post_discover_sales['discover_first_date']) == True)]
'''

"\npost_non_discover_new_customers = post_discover_sales[(~post_discover_sales['Email'].isin(pre_discover_sales['Email'])) \n                                                  &\n                                                  (pd.isnull(post_discover_sales['discover_first_date']) == True)]\n"

In [98]:
'''
# filter ALL sales_data_clean for new users who have tried discover
# sales_data_clean[(sales_data_clean['used_discover_already'] == 'Used Discover') & (sales_data_clean['Email'] == 'charmaine_287@hotmail.com')]
discover_users_sales = sales_data_clean[ pd.isnull(sales_data_clean['discover_first_date']) == False]
non_discover_users_sales = sales_data_clean[ pd.isnull(sales_data_clean['discover_first_date']) == True]
'''


"\n# filter ALL sales_data_clean for new users who have tried discover\n# sales_data_clean[(sales_data_clean['used_discover_already'] == 'Used Discover') & (sales_data_clean['Email'] == 'charmaine_287@hotmail.com')]\ndiscover_users_sales = sales_data_clean[ pd.isnull(sales_data_clean['discover_first_date']) == False]\nnon_discover_users_sales = sales_data_clean[ pd.isnull(sales_data_clean['discover_first_date']) == True]\n"

In [102]:
discover_taker_sales = sales_data_clean[sales_data_clean['discover_first_date'].isnull() == False] 
discover_taker_dates = discover_taker_sales[['Email', 'Created at']].drop_duplicates()
discover_taker_dates.sort_values('Created at', inplace=True)
discover_taker_dates['Previous Transaction'] = discover_taker_dates.groupby(['Email'])['Created at'].shift()
discover_taker_dates['Days Between'] = discover_taker_dates['Created at'] - discover_taker_dates['Previous Transaction']
discover_taker_dates['Days Between Int'] = (discover_taker_dates['Days Between'].dropna()/ np.timedelta64(1, "D")).astype(int)
discover_taker_dates['Buy Count'] = discover_taker_dates.groupby(['Email'])['Created at'].cumcount()+1
discover_taker_dates.to_excel("discover_taker_dates.xlsx")
discover_taker_dates['Days Between'].mean()


Timedelta('58 days 19:31:27.861773')

In [103]:
non_discover_taker_sales = sales_data_clean[sales_data_clean['discover_first_date'].isnull() == True] 
non_discover_taker_dates = non_discover_taker_sales[['Email', 'Created at']].drop_duplicates()
non_discover_taker_dates.sort_values('Created at', inplace=True)
non_discover_taker_dates['Previous Transaction'] = non_discover_taker_dates.groupby(['Email'])['Created at'].shift()
non_discover_taker_dates['Days Between'] = non_discover_taker_dates['Created at'] - non_discover_taker_dates['Previous Transaction']
non_discover_taker_dates['Days Between Int'] = (non_discover_taker_dates['Days Between'].dropna()/ np.timedelta64(1, "D")).astype(int)
non_discover_taker_dates['Buy Count'] = non_discover_taker_dates.groupby(['Email'])['Created at'].cumcount()+1
non_discover_taker_dates.to_excel("non_discover_taker_dates.xlsx")
non_discover_taker_dates['Days Between'].mean()


Timedelta('69 days 03:13:42.868776')

In [17]:
apple = 100.96996
print('{0:.2f}%'.format(apple))

100.97%
